In [1]:
import pandas as pd

# Schritt 1: Daten laden
print("Schritt 1: Daten laden")

# Dateinamen definieren
file_name = 'delivery_data.csv'

try:
    # CSV-Datei laden, Semikolon als Trennzeichen verwenden
    df = pd.read_csv(file_name, delimiter=';')

    # Validierung: Überprüfen, ob die Daten geladen wurden
    print(f"Daten aus '{file_name}' erfolgreich geladen.")
    print("\n--- Erste 5 Zeilen der Daten (Validierung) ---")
    print(df.head())

    print("\n--- Daten-Typen und fehlende Werte (Validierung) ---")
    df.info()

except FileNotFoundError:
    print(f"Fehler: Die Datei '{file_name}' wurde nicht gefunden.")
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")

Schritt 1: Daten laden
Daten aus 'delivery_data.csv' erfolgreich geladen.

--- Erste 5 Zeilen der Daten (Validierung) ---
   order_id   material_type  quantity supplier_location destination_location  \
0         1  Fertigprodukte       839          Mannheim              Leipzig   
1         2      Elektronik       782             Paris  Mülheim an der Ruhr   
2         3        Bauteile        52        Oberhausen               Erfurt   
3         4       Rohstoffe       650     Gelsenkirchen                Mainz   
4         5       Rohstoffe       839   Mönchengladbach             Nürnberg   

   order_date weather_conditions  holiday_season  distance_km transport_mode  \
0  2023-06-20               Klar           False       344.98        Schiene   
1  2023-05-14         Regnerisch           False       431.50         Straße   
2  2023-12-23              Sturm           False       294.96         Straße   
3  2023-09-26              Nebel           False       187.69         Straße 

In [2]:
import pandas as pd
import numpy as np

print("Schritt 2: Daten analysieren")

# Daten laden (erneut, da Colab-Zellen-Kontext)
try:
    df = pd.read_csv('delivery_data.csv', delimiter=';')
except Exception as e:
    print(f"Fehler beim Laden der Daten: {e}")
    # Hier würden wir normalerweise abbrechen, wenn die Daten nicht geladen werden können
    raise e

# --- 1. Datentyp 'order_date' korrigieren ---
# (Wie in Schritt 1 bei df.info() gesehen, war 'order_date' ein 'object')
print("\n--- 1. Datentyp-Korrektur ('order_date') ---")
try:
    df['order_date'] = pd.to_datetime(df['order_date'])
    print("Datentyp 'order_date' erfolgreich in 'datetime' umgewandelt.")
except Exception as e:
    print(f"Fehler bei der Datumsumwandlung: {e}")

# --- 2. Statistische Kennzahlen für NUMERISCHE Merkmale ---
# (Wie vom Nutzer gefordert: Mittelwert, Std, Min, Max)
print("\n--- 2. Statistische Kennzahlen (Numerische Merkmale) ---")
# Wir nutzen .describe() für alle numerischen Typen
# 'order_id' ist zwar numerisch, aber nur ein Identifikator.
# Wir filtern die relevanten Spalten, die der Nutzer im Prompt genannt hat + Zielvariable
numerical_features = ['quantity', 'distance_km', 'delivery_time_days']
print(df[numerical_features].describe())

# --- 3. Analyse der KATEGORISCHEN Merkmale ---
print("\n--- 3. Analyse der Kategorischen Merkmale (inkl. Boolean) ---")
# 'include=['object', 'bool']' zeigt Statistiken für nicht-numerische Daten
# (Anzahl, Eindeutige Werte, Häufigster Wert, Häufigkeit)
try:
    print(df.describe(include=['object', 'bool']))
except Exception as e:
    print(f"Fehler bei der Analyse der kategorischen Merkmale: {e}")

print("\n--- Validierung der Datentypen nach Korrektur ---")
df.info()

Schritt 2: Daten analysieren

--- 1. Datentyp-Korrektur ('order_date') ---
Datentyp 'order_date' erfolgreich in 'datetime' umgewandelt.

--- 2. Statistische Kennzahlen (Numerische Merkmale) ---
           quantity   distance_km  delivery_time_days
count  15000.000000  15000.000000        15000.000000
mean     498.581067   4651.110221           21.823400
std      288.411239   5127.007721           26.116256
min        1.000000      4.240000            1.000000
25%      249.000000    310.840000            4.600000
50%      497.000000    997.020000            9.900000
75%      746.000000   8872.020000           33.800000
max      999.000000  18758.880000          367.700000

--- 3. Analyse der Kategorischen Merkmale (inkl. Boolean) ---
       material_type supplier_location destination_location  \
count          15000             15000                15000   
unique             5                74                   74   
top       Elektronik            Berlin               Mumbai   
freq 

In [3]:
import pandas as pd
import altair as alt

print("Schritt 3: Daten visualisieren")

# --- 1. Daten laden ---
try:
    df = pd.read_csv('delivery_data.csv', delimiter=';')
except Exception as e:
    print(f"Fehler beim Laden der Daten: {e}")
    raise e

# --- 2. Datenbereinigung (NaN) ---
# Fokussieren auf die relevanten Spalten
relevant_cols = ['delivery_time_days', 'route_type', 'weather_conditions']
df_cleaned = df[relevant_cols].copy()

# Überprüfen auf NaN (obwohl wir von Schritt 1 wissen, dass es keine gibt,
# setzen wir die Anforderung um)
initial_rows = len(df_cleaned)
df_cleaned.dropna(inplace=True)
cleaned_rows = len(df_cleaned)

if initial_rows > cleaned_rows:
    print(f"{initial_rows - cleaned_rows} Zeilen mit NaN-Werten entfernt.")
else:
    print("Keine NaN-Werte in den relevanten Spalten gefunden. (Gute Praxis)")

# --- 3. & 4. Datenaggregation und Aufbereitung ---
# Berechne die durchschnittliche Transportzeit pro Routentyp und Wetter
try:
    df_agg = df_cleaned.groupby(['route_type', 'weather_conditions'])['delivery_time_days'].mean().reset_index()
    # Runden des Mittelwerts für eine sauberere Anzeige im Tooltip
    df_agg['delivery_time_days'] = round(df_agg['delivery_time_days'], 2)
    print("\nDaten erfolgreich aggregiert (Durchschnittliche Transportzeit):")
    print(df_agg.head())
except Exception as e:
    print(f"Fehler bei der Datenaggregation: {e}")
    raise e

# --- 5. Diagramm-Erstellung (Gruppiertes Säulendiagramm) ---
print("\nErstelle gruppiertes Säulendiagramm...")
try:
    chart = alt.Chart(df_agg).mark_bar().encode(
        # X-Achse (innerhalb der Gruppe): Wetter. Achsentitel entfernen.
        x=alt.X('weather_conditions', axis=None),

        # Y-Achse: Durchschnittliche Lieferzeit
        y=alt.Y('delivery_time_days', title='Durchschnittl. Transportzeit (Tage)'),

        # Farbe: Wird auch für die Gruppierung (Wetter) verwendet
        color=alt.Color('weather_conditions', title='Wetter'),

        # Spalten (Hauptgruppierung): Routentyp.
        # Titel und Labels nach unten verschoben für bessere Lesbarkeit.
        column=alt.Column(
            'route_type',
            title='Routentyp',
            header=alt.Header(titleOrient="bottom", labelOrient="bottom")
        ),

        # Tooltip für Interaktivität
        tooltip=['route_type', 'weather_conditions', 'delivery_time_days']
    ).interactive()

    # --- 6. Speichern ---
    chart_path = 'delivery_time_by_route_weather.json'
    chart.save(chart_path)
    print(f"Diagramm erfolgreich als '{chart_path}' gespeichert.")

except Exception as e:
    print(f"Fehler bei der Diagrammerstellung mit Altair: {e}")

Schritt 3: Daten visualisieren
Keine NaN-Werte in den relevanten Spalten gefunden. (Gute Praxis)

Daten erfolgreich aggregiert (Durchschnittliche Transportzeit):
         route_type weather_conditions  delivery_time_days
0  domestic_germany               Klar                4.11
1  domestic_germany              Nebel                4.27
2  domestic_germany         Regnerisch                4.04
3  domestic_germany             Schnee                4.57
4  domestic_germany              Sturm                4.71

Erstelle gruppiertes Säulendiagramm...
Diagramm erfolgreich als 'delivery_time_by_route_weather.json' gespeichert.


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

print("Schritt 4: KI-Modell (Regression) trainieren")

# --- 1. Daten laden und vorbereiten ---
try:
    df = pd.read_csv('delivery_data.csv', delimiter=';')
except Exception as e:
    print(f"Fehler beim Laden der Daten: {e}")
    raise e

# --- Feature Engineering (Schritt 2 der Checkliste) ---
# Wir wandeln 'order_date' in nützlichere numerische Features um.
# Das Modell kann mit "Monat" und "Wochentag" oft mehr anfangen.
try:
    df['order_date'] = pd.to_datetime(df['order_date'])
    df['order_month'] = df['order_date'].dt.month
    df['order_day_of_week'] = df['order_date'].dt.dayofweek
    # Die ursprüngliche Datumsspalte wird nicht mehr benötigt
    df = df.drop('order_date', axis=1)
    print("Feature Engineering für 'order_date' abgeschlossen.")
except Exception as e:
    print(f"Fehler beim Feature Engineering: {e}")

# --- 2. & 4. & 5. Preprocessing und Pipeline definieren ---

# Definition der Spalten (Features)
# HINWEIS: 'order_date' ist jetzt 'order_month' und 'order_day_of_week'
# Diese 11 Features entsprechen den 10 ursprünglichen Eingabefeldern
numeric_features = ['quantity', 'distance_km']
categorical_features = [
    'material_type', 'supplier_location', 'destination_location',
    'transport_mode', 'route_type', 'weather_conditions',
    'order_month', 'order_day_of_week' # Behandeln wir als kategorial
]
boolean_feature = ['holiday_season'] # 'holiday_season'

# Definition der Zielvariable (Label)
label = 'delivery_time_days'

# Erstellen der Preprocessing-Pipelines für verschiedene Datentypen
# Numerisch: Skalieren (Mittelwert 0, Varianz 1)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Kategorial: One-Hot-Encoding (Erzeugt Dummy-Variablen)
# handle_unknown='ignore' sorgt dafür, dass das Modell bei neuen,
# im Training unbekannten Werten (z.B. neue Stadt) keinen Fehler wirft.
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Zusammenbau des Preprocessors mit ColumnTransformer
# ('bool', 'passthrough', boolean_feature) -> 'holiday_season' (True/False)
# wird unverändert (als 0/1) durchgelassen.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bool', 'passthrough', boolean_feature)
    ],
    remainder='drop' # Ignoriert andere Spalten (wie order_id)
)

# Definition des Modells
regressor_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# Erstellen der vollständigen Pipeline (Preprocessing + Modell)
# Dies ist das Artefakt, das wir in Schritt 5 speichern werden.
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor_model)
])

print("ML-Pipeline erfolgreich definiert.")

# --- 3. Datentrennung ---
# Wir definieren X (alle Features) und y (das Label)
# Wir verwenden die 10 ursprünglichen Features + die 2 neuen Datumsfeatures
all_features = numeric_features + categorical_features + boolean_feature
X = df[all_features]
y = df[label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Daten gesplittet: {len(X_train)} Trainingsdatensätze, {len(X_test)} Testdatensätze.")

# --- 5. Modelltraining ---
print("Starte Modelltraining...")
try:
    model_pipeline.fit(X_train, y_train)
    print("Modelltraining erfolgreich abgeschlossen.")
except Exception as e:
    print(f"Fehler während des Trainings: {e}")
    raise e

# --- 6. Modell-Evaluierung ---
print("\n--- Modell-Evaluierung ---")
# Vorhersage auf den Testdaten
y_pred = model_pipeline.predict(X_test)

# Berechnung der Metriken
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R-squared (R²) Score: {r2:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f} (Tage)")

# Validierung basierend auf R²
if r2 > 0.7:
    print("Validierung: Modellgenauigkeit (R² > 0.7) ist gut.")
elif r2 > 0.5:
    print("Validierung: Modellgenauigkeit (R² > 0.5) ist akzeptabel, könnte aber besser sein.")
else:
    print("Validierung: Modellgenauigkeit (R² < 0.5) ist niedrig. Ggf. mehr Feature Engineering nötig.")

Schritt 4: KI-Modell (Regression) trainieren
Feature Engineering für 'order_date' abgeschlossen.
ML-Pipeline erfolgreich definiert.
Daten gesplittet: 12000 Trainingsdatensätze, 3000 Testdatensätze.
Starte Modelltraining...
Modelltraining erfolgreich abgeschlossen.

--- Modell-Evaluierung ---
R-squared (R²) Score: 0.9804
Root Mean Squared Error (RMSE): 3.8176 (Tage)
Validierung: Modellgenauigkeit (R² > 0.7) ist gut.


In [5]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

print("Schritt 5: Modell speichern")

# --- 1. Kontext und Training (Vorbereitung zum Speichern) ---
# Damit diese Zelle robust ist, definieren wir die Pipeline neu
# und trainieren sie mit 100% der Daten (Best Practice).

try:
    df = pd.read_csv('delivery_data.csv', delimiter=';')
except Exception as e:
    print(f"Fehler beim Laden der Daten: {e}")
    raise e

# 1a. Feature Engineering (wie in Schritt 4)
try:
    df['order_date'] = pd.to_datetime(df['order_date'])
    df['order_month'] = df['order_date'].dt.month
    df['order_day_of_week'] = df['order_date'].dt.dayofweek
    df = df.drop('order_date', axis=1)
except Exception as e:
    print(f"Fehler beim Feature Engineering: {e}")

# 1b. Definition der Features und Pipeline (wie in Schritt 4)
numeric_features = ['quantity', 'distance_km']
categorical_features = [
    'material_type', 'supplier_location', 'destination_location',
    'transport_mode', 'route_type', 'weather_conditions',
    'order_month', 'order_day_of_week'
]
boolean_feature = ['holiday_season']
label = 'delivery_time_days'

all_features = numeric_features + categorical_features + boolean_feature

# Pipeline-Definition
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bool', 'passthrough', boolean_feature)
    ],
    remainder='drop'
)

regressor_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor_model)
])

# 1c. Training mit 100% der Daten
print("Trainiere das finale Modell mit 100% der Daten (Best Practice)...")
try:
    X_full = df[all_features]
    y_full = df[label]

    model_pipeline.fit(X_full, y_full)
    print("Training des finalen Modells abgeschlossen.")
except Exception as e:
    print(f"Fehler während des finalen Trainings: {e}")
    raise e

# --- 2. Modell speichern ---
model_filename = 'transport_model.joblib'
try:
    # Speichern der gesamten Pipeline (Preprocessor + Regressor)
    joblib.dump(model_pipeline, model_filename)

    # Validierung: Prüfen, ob die Datei existiert
    import os
    if os.path.exists(model_filename):
        print(f"\nValidierung: Pipeline erfolgreich als '{model_filename}' gespeichert.")
    else:
        print(f"\nFehler: Datei '{model_filename}' wurde nicht erstellt.")

except Exception as e:
    print(f"Fehler beim Speichern des Modells: {e}")

Schritt 5: Modell speichern
Trainiere das finale Modell mit 100% der Daten (Best Practice)...
Training des finalen Modells abgeschlossen.

Validierung: Pipeline erfolgreich als 'transport_model.joblib' gespeichert.


In [7]:
# --- Schritt 6: Benutzeroberfläche (Gradio) ---
# HINWEIS: Dieser Code muss in einer Umgebung ausgeführt werden,
# in der 'gradio' installiert ist (z.B. nach 'pip install gradio').

print("Schritt 6: Benutzeroberfläche entwickeln und mit Gradio hosten")

# --- 1. Imports ---
try:
    import gradio as gr
except ImportError:
    print("Fehler: 'gradio' ist nicht installiert.")
    print("Bitte führen Sie 'pip install gradio' aus und starten Sie das Skript erneut.")
    exit()

import pandas as pd
import numpy as np
import joblib
from datetime import datetime
import os

print("Gradio und notwendige Bibliotheken importiert.")

# --- 2. Modell laden (Einmalig beim Start) ---
MODEL_FILE = 'transport_model.joblib'
MODEL_PIPELINE = None
try:
    MODEL_PIPELINE = joblib.load(MODEL_FILE)
    print(f"Modell '{MODEL_FILE}' erfolgreich geladen.")
except FileNotFoundError:
    print(f"FATALER FEHLER: Modelldatei '{MODEL_FILE}' nicht gefunden.")
    print("Bitte Schritt 5 (Modell speichern) zuerst ausführen.")
except Exception as e:
    print(f"FATALER FEHLER: Modell konnte nicht geladen werden: {e}")

# Definition der Spalten, die das Modell (und die Validierung) erwartet
EXPECTED_COLUMNS = [
    "material_type", "quantity", "supplier_location", "destination_location",
    "distance_km", "transport_mode", "route_type", "order_date",
    "weather_conditions", "holiday_season"
]

# --- 3. Kernlogik (Die Vorhersage-Funktion) ---
def predict_batch_csv(uploaded_file):
    """
    Wird von Gradio aufgerufen. Validiert, verarbeitet und sagt
    die Transportzeiten für eine hochgeladene CSV-Datei voraus.
    """

    # Validierung 1: Modell geladen?
    if MODEL_PIPELINE is None:
        return None, None, "FEHLER: Das KI-Modell ist nicht geladen. App bitte neu starten."

    # Validierung 2: Datei hochgeladen?
    if uploaded_file is None:
        return None, None, "Fehler: Bitte eine CSV-Datei hochladen."

    # Pfad zur temporären Datei (Gradio 3+)
    temp_file_path = uploaded_file.name

    try:
        # 1. Daten laden (Robustes Parsen)
        df = None
        try:
            # Versuche Komma
            df = pd.read_csv(temp_file_path)
        except (pd.errors.ParserError, UnicodeDecodeError, IsADirectoryError):
            df = None # Setze zurück, falls erster Versuch fehlschlägt

        if df is None or len(df.columns) <= 1:
             try:
                # Versuche Semikolon (wie die Originaldatei)
                df = pd.read_csv(temp_file_path, delimiter=';')
             except Exception as e_inner:
                return None, None, f"CSV-Lesefehler. Bitte Komma (,) oder Semikolon (;) verwenden. Details: {e_inner}"

        if df is None:
             return None, None, "CSV-Datei konnte nicht gelesen werden."

        df_input_original = df.copy()

        # 2. Spaltenvalidierung
        uploaded_cols_original = set(df.columns)
        required_cols_set = set(EXPECTED_COLUMNS)

        if not required_cols_set.issubset(uploaded_cols_original):
            missing = list(required_cols_set - uploaded_cols_original)
            return None, None, f"Fehler: Fehlende Spalten: {missing}. Gefunden: {list(uploaded_cols_original)}"

        df = df[EXPECTED_COLUMNS]

        # 3. Datentyp-Validierung & Konvertierung
        df_processed = df.copy()

        try:
            df_processed['order_date'] = pd.to_datetime(df_processed['order_date'])
        except Exception:
            return None, None, "Fehler: Spalte 'order_date' (z.B. '2024-07-05') konnte nicht verarbeitet werden."

        try:
            df_processed['quantity'] = pd.to_numeric(df_processed['quantity'])
        except Exception:
            return None, None, "Fehler: Spalte 'quantity' muss numerisch sein."

        try:
            df_processed['distance_km'] = pd.to_numeric(df_processed['distance_km'])
        except Exception:
            return None, None, "Fehler: Spalte 'distance_km' muss numerisch sein."

        if df_processed['holiday_season'].dtype == 'object':
            df_processed['holiday_season'] = df_processed['holiday_season'].str.lower().str.strip().map({
                'true': True, 'false': False, '1': True, '0': False,
                'wahr': True, 'falsch': False, 'ja': True, 'nein': False
            }).fillna(False)

        df_processed['holiday_season'] = df_processed['holiday_season'].astype(bool)

        print("Validierung der Eingabedaten erfolgreich.")

        # 4. Feature Engineering
        df_processed['order_month'] = df_processed['order_date'].dt.month
        df_processed['order_day_of_week'] = df_processed['order_date'].dt.dayofweek
        print("Feature Engineering abgeschlossen.")

        # 5. Vorhersage
        predictions = MODEL_PIPELINE.predict(df_processed)
        print("Vorhersage erfolgreich durchgeführt.")

        # 6. Ergebnisse formatieren
        df_input_original['VORHERSAGE_Transportzeit_Tage'] = np.round(predictions, 2)

        # 7. Download-Datei erstellen
        download_path = "ergebnisse_transportvorhersage.csv"
        df_input_original.to_csv(download_path, index=False, sep=';', encoding='utf-8-sig')

        return df_input_original, download_path, "Vorhersage erfolgreich abgeschlossen."

    except Exception as e:
        if temp_file_path and os.path.exists(temp_file_path):
             os.remove(temp_file_path)
        print(f"Ein Fehler ist aufgetreten: {e}")
        return None, None, f"Allgemeiner Fehler: {str(e)}"
    finally:
        # Aufräumen: Lösche die temporäre Upload-Datei
        if temp_file_path and os.path.exists(temp_file_path) and os.path.isfile(temp_file_path):
             try:
                 os.remove(temp_file_path)
             except OSError as e_rm:
                 print(f"Fehler beim Löschen der temporären Datei {temp_file_path}: {e_rm}")


# --- 4. UI-Definition (Gradio Blocks) ---
csv_format_description = """
**Anleitung und CSV-Format:**

1.  Laden Sie eine CSV-Datei hoch (Trennzeichen: Komma `,` oder Semikolon `;`).
2.  Die Datei *muss* die folgenden 10 Spalten enthalten (Reihenfolge egal, Namen müssen exakt stimmen):
    - `material_type` (Text, z.B. "Elektronik")
    - `quantity` (Zahl, z.B. 1000)
    - `supplier_location` (Text, z.B. "Berlin")
    - `destination_location` (Text, z.B. "Hamburg")
    - `distance_km` (Zahl, z.B. 280.0)
    - `transport_mode` (Text, z.B. "Straße")
    - `route_type` (Text, z.B. "domestic_germany")
    - `order_date` (Datum/Text, z.B. "2024-07-05")
    - `weather_conditions` (Text, z.B. "Regnerisch")
    - `holiday_season` (Boolean, z.B. `TRUE`/`FALSE`, `1`/`0`, `wahr`/`falsch`)
3.  Klicken Sie auf "Vorhersage starten".
"""

# Überprüfen, ob das Modell geladen wurde, bevor die UI definiert wird
if MODEL_PIPELINE is not None:
    with gr.Blocks(theme=gr.themes.Soft()) as iface:
        gr.Markdown("# KI-Modell: Vorhersage von Transportzeiten 🚚")
        gr.Markdown(csv_format_description)

        with gr.Row():
            csv_in = gr.File(label="CSV-Datei hochladen", file_types=[".csv"])
            btn_predict = gr.Button("Vorhersage starten", variant="primary")

        gr.Markdown("---")
        gr.Markdown("### Ergebnisse")

        status_text = gr.Textbox(label="Status / Fehler", interactive=False)
        results_table = gr.Dataframe(label="Ergebnisse (Eingabe + Vorhersage)", interactive=False)
        results_file = gr.File(label="Ergebnis-CSV herunterladen")

        btn_predict.click(
            fn=predict_batch_csv,
            inputs=[csv_in],
            outputs=[results_table, results_file, status_text]
        )

# --- 5. App starten ---
if MODEL_PIPELINE is not None and 'iface' in locals():
    print("\n--- Gradio App wird gestartet ---")
    print("Klicken Sie auf den Link (z.B. [http://127.0.0.1:7860](http://127.0.0.1:7860)) oder den Public Link (share=True).")

    # share=True erstellt einen öffentlichen Link (nützlich in Colab)
    # debug=True zeigt detailliertere Fehler in der Konsole an
    iface.launch(share=True, debug=True)
else:
    print("\n--- Gradio App NICHT gestartet ---")
    print("Das Modell konnte nicht geladen werden. Bitte beheben Sie den Fehler und führen Sie das Skript erneut aus.")

Schritt 6: Benutzeroberfläche entwickeln und mit Gradio hosten
Gradio und notwendige Bibliotheken importiert.
Modell 'transport_model.joblib' erfolgreich geladen.

--- Gradio App wird gestartet ---
Klicken Sie auf den Link (z.B. [http://127.0.0.1:7860](http://127.0.0.1:7860)) oder den Public Link (share=True).
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://14c599098c54361648.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Validierung der Eingabedaten erfolgreich.
Feature Engineering abgeschlossen.
Vorhersage erfolgreich durchgeführt.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://14c599098c54361648.gradio.live
